# Lab06 Решение

## Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

In [11]:
#Lab06 personal data
my_id = 286

In [12]:
#Lab06s personal data
k = 15
z = 3.291

In [13]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [14]:
sc

In [15]:
#sc.stop()

## Loading user data

In [16]:
# Loading from HDFS, make sure you put you data into hdfs first
user_data_src = sc.textFile("/labs/lab06data/ml-100k/u.data")

In [17]:
# Data lookup:
user_data_src.first()

In [18]:
user_data = user_data_src.map(lambda u: u.split("\t"))

In [19]:
user_data.take(5)

In [20]:
num_users = user_data.count()
num_users

In [21]:
%pylab inline

In [22]:
user_ages = user_data.map(lambda u: int(u[1]))
user_ages.take(5)

In [23]:
ages = user_ages.collect()
hist(ages, bins=10)

In [24]:
ages = user_ages.collect

## Loading movie data

In [26]:
movie_data_src = sc.textFile("/labs/lab06data/ml-100k/u.item")
movie_data_src.first()

In [27]:
movie_data_src.take(10)

In [28]:
movie_data = movie_data_src.map(lambda line: line.split("|"))
movie_data.take(5)

In [29]:
from datetime import datetime

def get_year(date_string):
    try:
        return int(datetime.strptime(date_string, "%d-%b-%Y").year)
    except:
        return 1899

In [30]:
movie_years = movie_data.map(lambda m: get_year(m[2]))

In [31]:
movie_years.take(10)

In [32]:
years = movie_years.collect()
hist(years,bins=100)

## Loading ratings data

In [34]:
# user id | item id | rating | timestamp
ratings_src = sc.textFile("/labs/lab06data/ml-100k/u.data")
ratings_src.first()

In [35]:
ratings_data = ratings_src.map(lambda r: r.split("\t"))
ratings_data.take(3)

In [36]:
ratings = ratings_data.map(lambda r: int(r[2])).collect()

In [37]:
hist(ratings)

In [38]:
hist(ratings,bins=[1,2,3,4,5,6])

In [39]:
# 0: user id | 1: item id | 2: rating | 3: timestamp
ratings_map = ratings_data.map(lambda r: (r[1], int(r[2])))

In [40]:
#item_id, rating
ratings_map.first()

#### 1. Количество человек n, поставивших рейтинг фильму

In [41]:
#kv = {item_id: 1}
n1_kv_m = ratings_data.map( lambda x: (x[1], 1) )

#kv = {item_id: sum}
n1 = n1_kv_m.reduceByKey( lambda pv,v: pv+v )

n1.take(5)

#### 2. Средний рейтинг фильма (сумма рейтингов фильма/количество человек, оценивних фильм)

In [42]:
#kv = {item_id: (rating, 1)}
n2_kv_m = ratings_data.map( lambda x: (x[1], (int(x[2]), 1)) )

#kv = {item_id: (rating_sum, count)}
n2_kv_r = n2_kv_m.reduceByKey( lambda pv,v: (pv[0]+v[0], pv[1]+v[1]) )
n2_kv_r.first()

In [43]:
from __future__ import division
n2 = n2_kv_r.map( lambda x: (x[0], x[1][0] / x[1][1]) )

n2.take(5)

#### 3. Количество человек m, оценивших фильм положительно. Оценки 4 и выше -- считаются положительными.

In [44]:
#kv = {item_id: (1 if rating >=4 else 0)}
n3_kv_m = ratings_data.map( lambda x: (x[1], 1 if int(x[2]) >=4 else 0) )
n3_kv_m.take(5)

In [45]:
#kv = {item_id: sum}
n3 = n3_kv_m.reduceByKey( lambda pv,v: pv+v )

In [46]:
n3.take(5)

#### 4. Доля людей, оценивших фильм положительно (пункт 3/пункт 1 или m/n).¶

In [47]:
#kv = {item_id: (n1, n3) }
n4_kv_m = n1.join(n3)
n4 = n4_kv_m.map( lambda x: (x[0], x[1][1] / x[1][0]) )
n4.take(5)

#### 5. Глобальное среднее по всему датасету. Сумма всех оценок по всем фильмам/Количество всех оценок по всем фильмам

In [48]:
#kv = {rating}
n5_kv_m = ratings_data.map( lambda x: x[2] )

n5 = n5_kv_m.reduce( lambda pv,v: int(pv)+int(v) ) / n5_kv_m.count()
n5

#### 6. Оценку, поправленную на нехватку данных:

In [49]:

#n2_kv_r = {item_id: (rating_sum, count)}
#n2_kv_r.take(5)
n6 = n2_kv_r.map( lambda x: (x[0], (x[1][0] + k * n5) / (x[1][1] + k) ) )

In [50]:
n6.take(5)

#### 7. Нижнюю и верхнюю границы доверительного интервала оценки (Wilson score interval) из лекции с заданным уровнем доверия.

In [51]:
#kv = {item_id: (n1, n4) }
n7_kv_m = n1.join(n4)
n7 = n7_kv_m.map( lambda x: ( x[0], (1 / (1 + z**2/x[1][0])) * (x[1][1] + z**2/(2*x[1][0]) - z*(x[1][1]*(1-x[1][1])/x[1][0] + z**2/(4*x[1][0]**2))**0.5 ) ) )



In [52]:
n7.take(5)

#### Рекомендовать топ 10 фильмов (если рейтинги совпадают, то сортировать по алфавиту названий фильмов от A до Z):

#### 1. По откликам (пункт 1) - поле “top10_rates”

In [53]:
#kv = {item_id: (name, n1) }

movie_data_n1 = movie_data.join(n1)
top10_rates = [tp[0] for tp in movie_data_n1.top(10, lambda x: x[1][1])]
movie_data_n1.top(10, lambda x: x[1][1])

#### 2. По среднему рейтингу (пункт 2) - поле “top10_average”

In [54]:
top10_average = movie_data.join(n2).sortBy(lambda x: (-x[1][1], x[1][0])).keys().take(10)
movie_data.join(n2).sortBy(lambda x: (-x[1][1], x[1][0])).take(10)

#### 3. По среднему рейтингу с регуляризацией k (пункт 6) - поле “top10_rating”

In [55]:
top10_rating = movie_data.join(n6).sortBy(lambda x: (-x[1][1], x[1][0])).keys().take(10)
movie_data.join(n6).sortBy(lambda x: (-x[1][1], x[1][0])).take(10)

#### 4. По нижней границе доверительного интервала Wilson (пункт 7) - поле “top10_lower”

In [56]:
top10_lower = movie_data.join(n7).sortBy(lambda x: (-x[1][1], x[1][0])).keys().take(10)
movie_data.join(n7).sortBy(lambda x: (-x[1][1], x[1][0])).take(10)

#### Для выданного id фильма:

#### 1. Построить распределения оценок. Получится таблица, где в первом столбце стоят значения оценки (1, 2, 3, 4, 5), а во втором количество человек, поставивших соответствующую оценку - поле “hist_film”


In [57]:

hist_film_kv_f = ratings_data.filter(lambda x: (x[1] == str(my_id)))
hist_film_kv_m = hist_film_kv_f.map(lambda x: (x[2], 1))
hist_film_kv_r = hist_film_kv_m.reduceByKey(lambda x,y: x+y)
hist_film = [i[1] for i in hist_film_kv_r.sortByKey().collect()]

#### 2. Чтобы понять какие оценки предпочитают ставить пользователи Movielens, постройте аналогичное предыдущему пункту 1 распределение оценок только уже для всего датасета - поле “hist_all”

In [58]:

hist_all_kv_m = ratings_data.map(lambda r: (r[2], 1))
hist_all_kv_r = hist_all_kv_m.reduceByKey(lambda x,y: x+y)
hist_all = [i[1] for i in hist_all_kv_r.sortByKey().collect()]

In [59]:
result_json={
            "top10_rates": top10_rates,
            "top10_average": top10_average,
            "top10_rating": top10_rating,
            "top10_lower": top10_lower,
            "hist_film": hist_film,
            "hist_all": hist_all
            }

In [60]:
import json
with open("lab06.json", "w") as j:
    json.dump(result_json,j)

In [61]:
#superachievka
import json
with open("lab06s.json", "w") as j:
    json.dump(result_json,j)

In [62]:
sc.stop()